# Create initial and boundary conditions for BGC of ACCESS-OM2*

### A few notes:

0. Directory structure
    - `input` contains input data.
    - `tmp` is where the regridded data will be stored initially (to be deleted manually after regridding is finished).
    - `1deg, 025deg, 01deg` are where the regridded data used for model simulations will be stored (permanent locations).
0. To run ACCESS-OM2* with BGC, the following files are required:
    - bgc_param.nc (parameters)
    - co2_iaf.nc or co2_ryf.nc (atmospheric co2 concentrations)
    - csiro_bgc.res.nc (initial conditions)
    - csiro_bgc_sediment.res.nc (initial sediment conditions)
    - dust.nc (dust deposition for iron)

0. In addition, the following files need to be updated by adding extra coupling tracers for ice BGC:
    - i2o.nc
    - o2i.nc

0. There are a few options for BGC initial conditions. The current version of this script supports generation of initial conditions either from:
    - access-om2-1deg_omip2_cycle5 (adic,dic,alk)
    - WOA13v2 (no3,o2; OMIP protocol)
    - GLODAPv2 2016b (no3,o2,adic,dic,alk)
    - FeMIP median (fe; mol L-1)
    - 0.01 mmol m-3 is given to phy,zoo,det,caco3.

0. Before regridding, input data are extrapolated over land to avoid missing values over the regridded ocean in case slight changes in the model bathymetry occurs in the future. This is done using `cdo fillmiss`.

0. WOMBAT wants these fields in the units of mmol m-3 for all tracers except for iron (umol m-3). Therefore, appropriate unit conversion is needed if the input data are provided in different units. 

0. This script should be run using `gadi_jupyter` by requesting the maximum memory for a whole node i.e., `192 GB`.

# Load relevant modules

In [1]:
%matplotlib inline
from scipy.interpolate import griddata,interpn
import numpy as np
import xarray as xr
import xesmf as xe
#from dask.distributed import Client,LocalCluster
#client = Client()
#client

# Create `csiro_bgc.res.nc`.

- The input options `input_list` are:
    - IRON = FEMIP median iron
    - TAlk,TCO2,PI_TCO2,NO3,oxygen = GLODAPv2 2016b
    - no3,o2 = WOA2013v2
    - dic,adic,alk = 1deg_jra55_iaf_omip2_cycle5_restart304
- `wombat_list` contains the names of variables that WOMBAT wants. The choices are `fe,dic,adic,alk,no3,o2`.

- `wombat_list` must match the order of `input_list`. For example, if `input_list = ['NO3','oxygen']`, then `wombat_list = ['no3','o2']`

- `res_list` is the output resolution you want to regrid to.

- mean/min/max of both input and output data are printed to make sure that regridded values are reasonable and contains no NaNs.

In [2]:
res_list = ['1','025','01']
input_list = ['NO3','oxygen','PI_TCO2','TAlk','TCO2','IRON']#,'no3','o2','dic','adic','alk']
wombat_list = ['no3','o2','dic'  ,'alk' ,'adic',    'fe']# ,'no3','o2','dic','adic','alk']

for ir in range(np.size(res_list)):
    
    for it in range(np.size(wombat_list)):

        if input_list[it] == 'IRON':
            ds_in = xr.open_dataset('/g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/input/FEMIP_model_median_iron_2016_fillmiss.nc')
            #rename the coordinates to the MOM coordinate names to enable interpolation.
            ds_in = ds_in.rename(({'LON':'xt_ocean', 'LAT':'yt_ocean', 'DEPTH':'st_ocean'}))              
        elif input_list[it] == 'no3':
            ds_in = xr.open_dataset('/g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/input/woa13_nitrate_OMIPinit_fillmiss.nc',decode_times=False)
            ds_in = ds_in.rename(({'lon':'xt_ocean', 'lat':'yt_ocean', 'depth':'st_ocean'}))
            ds_in = ds_in.isel(time=0,drop=True)
        elif input_list[it] == 'o2':
            ds_in = xr.open_dataset('/g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/input/woa13_oxygen_OMIPinit_fillmiss.nc',decode_times=False)
            ds_in = ds_in.rename(({'lon':'xt_ocean', 'lat':'yt_ocean', 'depth':'st_ocean'}))
            ds_in = ds_in.isel(time=0,drop=True)
        elif input_list[it] in ['alk','adic','dic']:
            ds_in = xr.open_dataset('/g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/input/1deg_jra55_iaf_omip2_cycle5_restart304_csiro_bgc.res_fillmiss.nc')
            ds_in = ds_in.isel(time=0,drop=True)
        elif input_list[it] in ['TAlk','TCO2','PI_TCO2','NO3','oxygen']:
            ds_in = xr.open_dataset('/g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/input/GLODAPv2.2016b.'+input_list[it]+'_fillmiss.nc')
            ds_in = ds_in.rename(({'lon':'xt_ocean', 'lat':'yt_ocean', 'depth_surface':'st_ocean'}))
            #the vertical coordinate (depth_surface) in GLODAPv2.2016b is wrong (it is index, not depth values).
            #derive depth from FEMIP iron data (they are interpolated onto the same depths).
            ds_tmp = xr.open_dataset('/g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/input/FEMIP_model_median_iron_2016_fillmiss.nc')
            ds_in['st_ocean'] = ds_tmp['DEPTH']    
        ds_in = ds_in.rename({input_list[it]:wombat_list[it]})

            
        #first convert the lon coordiante system to the standard system (0-360) if not already in the standard system (e.g. -180 to 180)
        #retain lon values less than 360, otherwiise subtract 360
        ds_in['xt_ocean'] = ds_in['xt_ocean'].where(ds_in['xt_ocean']<360,ds_in['xt_ocean']-360)
        #retain lon values greater than 0, otherwise add 360
        ds_in['xt_ocean'] = ds_in['xt_ocean'].where(ds_in['xt_ocean']>0,ds_in['xt_ocean']+360)

        #now convert to the MOM lon coordinate system, which spans from -280 to 80 (yes, a bit weird)
        #retain lon values less than 80, otherwise subtract 360
        ds_in['xt_ocean'] = ds_in['xt_ocean'].where(ds_in['xt_ocean']<80,ds_in['xt_ocean']-360)
        #retain lon values greater than -280, otherwise add 360
        ds_in['xt_ocean'] = ds_in['xt_ocean'].where(ds_in['xt_ocean']>-280,ds_in['xt_ocean']+360)

        
        #read target grid
        if res_list[ir] == '1':
            ds_tar = xr.open_dataset('/g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/1deg/ocean_grid_xyz.nc')
            ds_gri = xr.open_dataset('/g/data/ik11/grids/ocean_grid_10.nc')
        elif res_list[ir] == '025':
            ds_tar = xr.open_dataset('/g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/025deg/ocean_grid_xyz.nc')
            ds_gri = xr.open_dataset('/g/data/ik11/grids/ocean_grid_025.nc')        
        elif res_list[ir] == '01':
            ds_tar = xr.open_dataset('/g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/01deg/ocean_grid_xyz.nc')
            ds_gri = xr.open_dataset('/g/data/ik11/grids/ocean_grid_01.nc')        
        #rename to lon and lat so that xesmf understands
        ds_gri = ds_gri.rename({'geolon_t': 'lon', 'geolat_t': 'lat'})

        #vertical 1-d interpolation first
        ar_zz = np.zeros((np.size(ds_tar['st_ocean']),np.size(ds_in['yt_ocean']),np.size(ds_in['xt_ocean'])))
        ar_in = ds_in[wombat_list[it]].values
        for ix in range(np.size(ar_zz,2)):
            for iy in range(np.size(ar_zz,1)):
                #fill_value=None will extrapolate
                ar_zz[:,iy,ix] = interpn((ds_in['st_ocean'].values,),ar_in[:,iy,ix],ds_tar['st_ocean'].values,
                                        method="linear",fill_value=None,bounds_error=False)         
        ds_zz = ds_in.copy()
        ds_zz = ds_zz.isel(st_ocean=0,drop=True)
        ds_zz = ds_zz.assign_coords(st_ocean=ds_tar['st_ocean'])
        if input_list[it] in ['adic','alk','dic']:
            ds_gri_1deg = xr.open_dataset('/g/data/ik11/grids/ocean_grid_10.nc')
            ds_zz['lon'] = (['yt_ocean','xt_ocean'],ds_gri_1deg['geolon_t'].values)
            ds_zz['lat'] = (['yt_ocean','xt_ocean'],ds_gri_1deg['geolat_t'].values)
        else:        
            ds_zz = ds_zz.rename({'xt_ocean': 'lon', 'yt_ocean': 'lat'})
        ds_zz[wombat_list[it]] = (['st_ocean','yt_ocean','xt_ocean'],ar_zz)           
        #use bilinear. i compared with nearest_s2d but the differences are minimal.
        regridder = xe.Regridder(ds_zz,ds_gri,method='bilinear',periodic=True)
        regridder        
        ds_tar = ds_tar.rename({'temp':wombat_list[it]})
        for iz in range(np.size(ds_zz['st_ocean'])):
            ds_tar[wombat_list[it]][:,iz,:,:] = regridder(ds_zz[wombat_list[it]].isel(st_ocean=iz))
        #some interpolated values are NaNs, so interpolate_na with the following options to fill in NaNs.
        #dim='st_ocean' is essential. 'xt_ocean' produces NaNs.
        #method='nearest' is essential. 'linear' gives extreme values.
        #fill_value='extrapolate' is essential. without it, still NaNs exist.
        ds_tar[wombat_list[it]] = ds_tar[wombat_list[it]].interpolate_na(dim='st_ocean',method='nearest',fill_value="extrapolate")
        print(['input: ',ds_in[wombat_list[it]].mean(),ds_in[wombat_list[it]].min(),ds_in[wombat_list[it]].max()])
        print(['output: ',ds_tar[wombat_list[it]].mean(skipna=False),ds_tar[wombat_list[it]].min(skipna=False),ds_tar[wombat_list[it]].max(skipna=False)])
        #set negative values to zero (negative values exist in NO3 input) 
        ds_tar[wombat_list[it]] = ds_tar[wombat_list[it]].where(ds_tar[wombat_list[it]]>0,0)

        #unit conversion relevant for fe and o2
        if wombat_list[it] == 'fe':
            #conversion from mol l-1 to umol m-3
            ds_tar[wombat_list[it]] = ds_tar[wombat_list[it]]*1e9
        elif wombat_list[it] == 'o2':
            #conversion from ml l-1 to mmol m-3
            ds_tar[wombat_list[it]] = ds_tar[wombat_list[it]]*44.661
        elif input_list[it] in ['TAlk','TCO2','PI_TCO2','NO3','oxygen']:
            #conversion from umol kg-1 to mmol m-3
            ds_tar[wombat_list[it]] = ds_tar[wombat_list[it]]*1.025            
        
        #rename to what wombat wants.
        ds_tar = ds_tar.rename(({'xt_ocean':'xaxis_1', 'yt_ocean':'yaxis_1', 'st_ocean':'zaxis_1', 'time':'Time'}))    
        ds_tar.to_netcdf('/g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/tmp/'+wombat_list[it]+'_init_'+res_list[ir]+'deg.nc',
                         unlimited_dims='Time',
                         encoding={wombat_list[it]:{'zlib': True,'complevel': 9}})
        print(res_list[ir],input_list[it],'done')

/g/data/hh5/public/apps/miniconda3/envs/analysis3-21.07/lib/python3.9/site-packages/xarray/coding/times.py:527: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-21.07/lib/python3.9/site-packages/xesmf/frontend.py:522: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


['input: ', <xarray.DataArray 'dic' ()>
array(2156.25, dtype=float32), <xarray.DataArray 'dic' ()>
array(147.80667114), <xarray.DataArray 'dic' ()>
array(2478.56494141)]
['output: ', <xarray.DataArray 'dic' ()>
array(2144.346, dtype=float32), <xarray.DataArray 'dic' ()>
array(170.94919, dtype=float32), <xarray.DataArray 'dic' ()>
array(2574.3003, dtype=float32)]
01 dic done


### Combine the regridded output for each tracer into a single `csiro_bgc.res.nc`. Also add tracers that are set to constant (0.01 mmol m-3; phy,zoo,det,caco3).
- `dic_eq_adic = True` will set dic to be identical to adic (not using it for now).

In [3]:
res_list = ['01']#['1','025','01']
wombat_list = ['fe','o2', 'alk', 'no3','adic','dic']
dic_eq_adic = False

for ir in range(np.size(res_list)):
    
    for it in range(np.size(wombat_list)):
        ds_tar = xr.open_dataset('/g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/tmp/'+wombat_list[it]+'_init_'+res_list[ir]+'deg.nc')
        try:
            ds_tar_all = ds_tar_all.assign(ds_tar)
        except NameError:
            ds_tar_all = ds_tar.copy()
    ds_new = ds_tar[wombat_list[-1]].copy()
    if dic_eq_adic:
        ds_tar_all = ds_tar_all.assign(dic=ds_new)
    ds_new[:,:,:,:] = 0.01
    ds_tar_all = ds_tar_all.assign(phy=ds_new)
    ds_tar_all = ds_tar_all.assign(zoo=ds_new)
    ds_tar_all = ds_tar_all.assign(det=ds_new)
    ds_tar_all = ds_tar_all.assign(caco3=ds_new)
    ds_tar_all.to_netcdf('/g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/tmp/csiro_bgc.res_'+res_list[ir]+'deg.nc')
    del ds_tar_all
    print(res_list[ir],'done')

01 done


# Creation of `co2_iaf.nc` and `co2_ryf.nc`.

1. Create atmospheric pCO2 (boundary conditions). one for IAF and one for RYF.
1. prescribe annual mean (for saving storage) and global mean (for simplicity) data.
1. follow omip2 protocol until 2014. from 2015-2020, use the global average data from https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_annmean_gl.txt
1. For RYF, apply the mean of pco2 between 1990-1991 to be consistent with the RYF forcing. 

In [2]:
from cftime import DatetimeNoLeap

In [3]:
#combined atmospheric co2 data for input from 1850-2020.
#OMIP2 protocol for 1850-2014.
#NOAA GML DATA for 2015-2020.
atmco2 = [284.32, 284.45, 284.60, 284.73, 284.85, 284.94, 285.05, 285.20, 285.37, 285.54, 285.74, 285.93, 286.10, 286.27, 286.44, 286.61, 286.78, 286.95, 287.10, 287.22, 287.35, 287.49, 287.66, 287.86, 288.06, 288.29, 288.52, 288.75, 288.99, 289.22, 289.47, 289.74, 290.02, 290.26, 290.51, 290.80, 291.10, 291.41, 291.76, 292.11, 292.46, 292.82, 293.17, 293.48, 293.79, 294.08, 294.36, 294.65, 294.95, 295.30, 295.67, 296.01, 296.32, 296.65, 296.95, 297.29, 297.66, 298.10, 298.52, 298.94, 299.38, 299.83, 300.35, 300.91, 301.42, 301.94, 302.48, 303.01, 303.45, 303.81, 304.25, 304.60, 304.94, 305.27, 305.63, 305.81, 305.95, 306.18, 306.33, 306.49, 306.62, 306.82, 307.09, 307.40, 307.78, 308.23, 309.01, 309.76, 310.29, 310.85, 311.36, 311.81, 312.17, 312.39, 312.41, 312.38, 312.39, 312.49, 312.52, 312.63, 312.82, 313.01, 313.34, 313.73, 314.09, 314.41, 314.70, 314.99, 315.34, 315.81, 316.62, 317.30, 318.04, 318.65, 319.33, 319.82, 320.88, 321.48, 322.39, 323.25, 324.78, 325.40, 327.35, 329.91, 330.76, 330.83, 331.54, 333.35, 335.01, 336.60, 338.70, 340.06, 340.64, 342.27, 344.01, 345.46, 346.90, 348.77, 351.28, 352.89, 354.07, 355.35, 356.23, 356.92, 358.25, 360.24, 362.00, 363.25, 365.93, 367.84, 369.12, 370.67, 372.83, 375.41, 376.99, 378.91, 381.01, 382.60, 384.74, 386.28, 388.72, 390.94, 393.02, 395.72, 397.55, 399.65, 403.09, 405.22, 407.61, 410.07, 412.46]
yrsco2 = np.arange(1850,2020+1)

#co2 concentration used for RYF
co2_fix = np.mean(atmco2[1990-1850:1991+1-1850])

#IAF spans from 1957 to 2020
date_iaf = []
for it in np.arange(1957,2020+1):
    date_iaf.append(DatetimeNoLeap(it,7,1))

In [5]:
res_list = ['1','025','01']

for ir in range(np.size(res_list)):
    if res_list[ir] == '1':
        ds_tar = xr.open_dataset('/g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/1deg/ocean_grid_xyz.nc')
    elif res_list[ir] == '025':
        ds_tar = xr.open_dataset('/g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/025deg/ocean_grid_xyz.nc')
    elif res_list[ir] == '01':
        ds_tar = xr.open_dataset('/g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/01deg/ocean_grid_xyz.nc')

    ds_tar = ds_tar.drop_dims('st_ocean')
    ds_tar = ds_tar.drop_vars(['time','time_bounds'])
    ds_tar = ds_tar.rename(({'xt_ocean':'grid_x_T', 'yt_ocean':'grid_y_T'}))

    #RYF first
    #RYF only has two dates (the same concentration is used throughout the simulation.)    
    date_ryf = [
        DatetimeNoLeap(1800,7,1),
        DatetimeNoLeap(2101,7,1)
    ]    
    co2_3d = np.zeros((np.size(date_ryf),np.size(ds_tar.grid_y_T),np.size(ds_tar.grid_x_T)))
    #take the average of co2 between 1957-1958. fixed value is given to both RYF and nat_co2/dic
    co2_fix = np.mean(atmco2[1957-1850:1958+1-1850])
    co2_3d[:,:,:] = co2_fix
    ds_tar = ds_tar.assign_coords({"time":date_ryf})
    ds_tar['co2'] = xr.DataArray(co2_3d,dims=['time','grid_y_T','grid_x_T'])
    ds_tar.to_netcdf('/g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/tmp/co2_ryf_'+res_list[ir]+'deg.nc',
                     unlimited_dims='time',
                     encoding={'co2':{'_FillValue': None,'zlib': True,'complevel': 9},'time':{'_FillValue':None,'dtype':'float64'}})
    
    ds_tar = ds_tar.drop_vars(['co2','time'])
    
    #IAF next
    date_iaf = []
    for it in range(1957,2020+1):
        date_iaf.append(DatetimeNoLeap(it,7,1))    
    co2_3d = np.zeros((np.size(date_iaf),np.size(ds_tar.grid_y_T),np.size(ds_tar.grid_x_T)))
    for it in range(1957,2020+1):
        co2_3d[it-1957,:,:] = atmco2[it-1850]
    ds_tar = ds_tar.assign_coords({"time":date_iaf})
    ds_tar['co2'] = xr.DataArray(co2_3d,dims=['time','grid_y_T','grid_x_T'])
    ds_tar.to_netcdf('/g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/tmp/co2_iaf_'+res_list[ir]+'deg.nc',
                     unlimited_dims='time',
                     encoding={'co2':{'_FillValue': None,'zlib': True,'complevel': 9},'time':{'_FillValue':None,'dtype':'float64'}})
    print(res_list[ir],'done')

1 done
025 done
01 done


In [6]:
%%bash
cd /g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/tmp
for ir in 1 025 01
do
    for it in iaf ryf
    do
        filename=co2_${it}_${ir}deg.nc
        ncatted -a cartesian_axis,grid_x_T,c,c,"X" $filename
        ncatted -a cartesian_axis,grid_y_T,c,c,"Y" $filename
    done
    echo $ir,done
done

1,done
025,done
01,done


# Creation of `bgc_param.nc`

### Define parameter values.
- For the names and units of these parameters, see Table of Supporting Information of Hayashida et al. (2020): Background nutrient concentration determines phytoplankton bloom response to marine heatwaves. Global Change Biology.
- However, note that units given below may be different from Table of Hayashida et al. (2020), because the model wants them in SI. For example, `abio = 0.27 d-1` in Hayashida et al. (2020), but `abio = 3.125e-6 s-1` in `bgc_param.nc` below.
- The parameter values specifed below are identical to those of Jiehn et al. (2020): The Australian Earth System Model: ACCESS-ESM1.5. Journal of Southern Hemisphere Earth Systems Science.
- `nat_co2` is the fixed atmospheric co2 concentration (ppm) applied to `dic`. Typically this is set to a pre-industrial co2 value, as long as `dic` is representative of natural DIC. However, in some applications, you might prescribe different values. 

In [7]:
param_list = [
 ['abio',3.125e-6],
 ['alphabio',2.96296e-6],
 ['bbio',1.066],
 ['cbio',1],
 ['epsbio',1.85185e-5],
 ['fe_bkgnd',0.6],
 ['f_inorg',0.062],
 ['gam1bio',0.925],
 ['gam2bio',1.15741e-7],
 ['gbio',1.82292e-5],
 ['kcbio',21.2766],
 ['kwbio',25],
 ['k1bio',0.7],
 ['muecaco3',1.9838e-8],
 ['muecaco3_sed',4.05093e-8],
 ['muedbio',5.55556e-7],
 ['muedbio_sed',2.31481e-7],
 ['muepbio',4.62963e-7],
 ['muepsbio',2.89352e-6],
 ['muezbio',3.93519e-6],
 ['nat_co2',284.32],
 ['parbio',0.43],
 ['tscav_fe',3.17098e-8],
 ['wcaco3',6.94444e-5],
 ['wdetbio',2.77778e-4]
]

### Create data

In [8]:
res_list = ['1','025','01']

for ir in range(np.size(res_list)):
    if res_list[ir] == '1':
        ds_tar = xr.open_dataset('/g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/1deg/ocean_grid_xyz.nc')
    elif res_list[ir] == '025':
        ds_tar = xr.open_dataset('/g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/025deg/ocean_grid_xyz.nc')
    elif res_list[ir] == '01':
        ds_tar = xr.open_dataset('/g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/01deg/ocean_grid_xyz.nc')

    ds_tar['temp'] = ds_tar['temp'].isel(st_ocean=0,drop=True)
    ds_tar = ds_tar.drop_dims('st_ocean')
    ds_tar = ds_tar.drop_vars(['time_bounds'])
    ds_tar = ds_tar.rename(({'xt_ocean':'grid_x_T', 'yt_ocean':'grid_y_T'}))
    for it in range(np.size(param_list,0)):
        #where() is needed to remove the land mask.
        ds_tar[param_list[it][0]] = ds_tar['temp'].where(ds_tar['temp']>0,0)*0 + param_list[it][1]
    ds_tar = ds_tar.drop_vars('temp')
    comp = dict(_FillValue=-1e34, missing_value=-1e34)
    encoding = {var: comp for var in ds_tar.data_vars}
    ds_tar.to_netcdf('/g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/tmp/bgc_param_'+res_list[ir]+'deg.nc',
                     encoding=encoding,
                     unlimited_dims='time')#,encoding={'time':{'_FillValue':None,'dtype':'float64'}})
    print(res_list[ir],'done')

1 done
025 done
01 done


In [9]:
%%bash
cd /g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/tmp
for ir in 1 025 01
do
    filename=bgc_param_${ir}deg.nc
    ncatted -a cartesian_axis,grid_x_T,c,c,"X" $filename
    ncatted -a cartesian_axis,grid_y_T,c,c,"Y" $filename
    ncks -O -L 9 $filename $filename
    echo $ir,done
done

# Creation of `dust.nc`
1. The original source file for dust input is not available, so use the 1-degree interpolated file as input. the method is described in Mongin et al. (2011), 10.1016/j.dsr2.2011.06.001.

In [2]:
res_list = ['1','025','01']

for ir in range(np.size(res_list)):
    #read target grid
    if res_list[ir] == '1':
        ds_tar = xr.open_dataset('/g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/1deg/ocean_grid_xyz.nc')
        ds_gri = xr.open_dataset('/g/data/ik11/grids/ocean_grid_10.nc')
    elif res_list[ir] == '025':
        ds_tar = xr.open_dataset('/g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/025deg/ocean_grid_xyz.nc')
        ds_gri = xr.open_dataset('/g/data/ik11/grids/ocean_grid_025.nc')        
    elif res_list[ir] == '01':
        ds_tar = xr.open_dataset('/g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/01deg/ocean_grid_xyz.nc')
        ds_gri = xr.open_dataset('/g/data/ik11/grids/ocean_grid_01.nc')        
    #rename to lon and lat so that xesmf understands
    ds_gri = ds_gri.rename({'geolon_t': 'lon', 'geolat_t': 'lat'})

    ds_in = xr.open_dataset('/g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/input/dust.nc')
    ds_gri_in = xr.open_dataset('/g/data/ik11/grids/ocean_grid_10.nc')
    ds_gri_in = ds_gri_in.rename({'geolon_t': 'lon', 'geolat_t': 'lat'})
    
    ds_tar['temp'] = ds_tar['temp'].isel(st_ocean=0,drop=True)
    ds_tar = ds_tar.drop_dims('st_ocean')
    ds_tar = ds_tar.drop_vars(['time','time_bounds'])
    ds_tar = ds_tar.drop_vars('temp')
    ds_tar = ds_tar.assign_coords({"time":ds_in['time']})
    ds_tar['lon'] = (['yt_ocean','xt_ocean'],ds_gri['lon'].values)
    ds_tar['lat'] = (['yt_ocean','xt_ocean'],ds_gri['lat'].values)
    ds_in['lon'] = (['yt_ocean','xt_ocean'],ds_gri_in['lon'].values)
    ds_in['lat'] = (['yt_ocean','xt_ocean'],ds_gri_in['lat'].values)

    regridder = xe.Regridder(ds_gri_in,ds_gri,method='bilinear',periodic=True)
    ds_tar['dust'] = (['time','yt_ocean','xt_ocean'],regridder(ds_in['dust']).values)    
    ds_tar = ds_tar.rename(({'xt_ocean':'grid_x_T', 'yt_ocean':'grid_y_T'}))
    #some interpolated values are NaNs, so interpolate_na with the following options to fill in NaNs.
    #dim='st_ocean' is essential. 'xt_ocean' produces NaNs.
    #method='nearest' is essential. 'linear' gives extreme values.
    #fill_value='extrapolate' is essential. without it, still NaNs exist.
    ds_tar['dust'] = ds_tar['dust'].interpolate_na(dim='time',method='nearest',fill_value="extrapolate")
    print(['input: ',ds_in['dust'].mean(),ds_in['dust'].min(),ds_in['dust'].max()])
    print(['output: ',ds_tar['dust'].mean(),ds_tar['dust'].min(),ds_tar['dust'].max()])

    ds_tar.to_netcdf('/g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/tmp/dust_'+res_list[ir]+'deg.nc',
                     unlimited_dims='time',
                     encoding={'dust':{'zlib': True,'complevel': 1,
                                       '_FillValue':-1.e34, 'missing_value':-1.e34}})
    print(res_list[ir],'done')

/g/data/hh5/public/apps/miniconda3/envs/analysis3-21.07/lib/python3.9/site-packages/xesmf/frontend.py:590: UserWarning: Using dimensions ('grid_y_T', 'grid_x_T') from data variable dust as the horizontal dimensions for the regridding.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-21.07/lib/python3.9/site-packages/xesmf/frontend.py:522: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


['input: ', <xarray.DataArray 'dust' ()>
array(5.21357542e-08), <xarray.DataArray 'dust' ()>
array(0.), <xarray.DataArray 'dust' ()>
array(2.1554396e-05)]
['output: ', <xarray.DataArray 'dust' ()>
array(3.92675238e-08), <xarray.DataArray 'dust' ()>
array(0.), <xarray.DataArray 'dust' ()>
array(2.12758077e-05)]


# Creation of `csiro_bgc_sediment.res.nc`
1. for simplicity, initialise with zero values everywhere.
1. the x and y dimensions of the input file are not helpful (given indices instead of actual lon/lat values), so replace them with dust.nc for interpolation purposes.
1. This script relies on `csiro_bgc.res.nc`, so make sure to compute this after creating `csiro_bgc.res.nc`.

In [3]:
res_list = ['1','025','01']

for ir in range(np.size(res_list)):
    ds_in = xr.open_dataset('/g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/tmp/csiro_bgc.res_'+res_list[ir]+'deg.nc')
    ds_tar = ds_in['phy'].isel(zaxis_1=0,drop=True)
    ds_tar = ds_tar*0
    ds_tar = ds_tar.rename('caco3_sediment')
    
    ds_tar.to_netcdf('/g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/tmp/csiro_bgc_sediment.res_'+res_list[ir]+'deg.nc',unlimited_dims='Time',encoding={'Time':{'_FillValue':None,'dtype':'float64'}})
    print(res_list[ir],'done')

In [4]:
%%bash
cd /g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/tmp
for ir in 1 025 01
do
    filename=csiro_bgc_sediment.res_${ir}deg.nc
    ncap2 -O -s 'det_sediment=caco3_sediment' $filename $filename
    ncatted -a cartesian_axis,xaxis_1,c,c,"X" $filename
    ncatted -a cartesian_axis,yaxis_1,c,c,"Y" $filename
    ncks -O -L 9 $filename $filename
done

# Update `o2i.nc` and `i2o.nc`
- two bgc tracers added to each file.
- wind speed is added to `i2o.nc` for gas flux calculation.

In [1]:
%%bash
for ir in 1 025 01
do
    cd /g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/${ir}deg/
    ncap2 -O -s 'ssn_i=sst_i/sst_i' o2i.nc o2i.nc
    ncap2 -O -s 'ssalg_i=sst_i/sst_i' o2i.nc o2i.nc
    ncap2 -O -s 'wnd10_io=licefh_io' i2o.nc i2o.nc
    ncap2 -O -s 'nit_io=wnd10_io+0.1' i2o.nc i2o.nc
    ncap2 -O -s 'alg_io=wnd10_io+0.1' i2o.nc i2o.nc
done

# Copy the regridded files to their respective destinations.
1. the regridded files will be copied to each resoluiton diretory and given the same file names (so that no need to customise ocean/field_table for each resolution).

In [10]:
%%bash
cd /g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/tmp
for ir in 1 025 01
do
    for it in bgc_param co2_iaf co2_ryf dust csiro_bgc.res csiro_bgc_sediment.res
    do
        cp ${it}_${ir}deg.nc ../${ir}deg/${it}.nc
    done
done

cp: cannot stat 'csiro_bgc_sediment.res_1deg.nc': No such file or directory


# That's it! If you are happy with the output stored in `1deg, 025deg, 01deg`, then delete `tmp`.

# Pre-processing of input and reference grid data (NOT NECESSARY, as these data already exist, but keeping it for reproducibility).

In [ ]:
#Download GLODAPv2 2016b and OMIP archives of WOA13v2 (no3, o2) and FEMIP (fe)
#Extract GLODAPv2
%%bash
cd input
wget https://www.nodc.noaa.gov/archive/arc0107/0162565/1.1/data/0-data/mapped/GLODAPv2.2016b_MappedClimatologies.tar.gz
tar -xvf GLODAPv2.2016b_MappedClimatologies.tar.gz
wget https://vesg.ipsl.upmc.fr/thredds/fileServer/IPSLFS/jomce/OMIP/initconds/FEMIP_model_median_iron_2016.nc
wget https://vesg.ipsl.upmc.fr/thredds/fileServer/IPSLFS/jomce/OMIP/initconds/woa13_oxygen_OMIPinit.nc
wget https://vesg.ipsl.upmc.fr/thredds/fileServer/IPSLFS/jomce/OMIP/initconds/woa13_nitrate_OMIPinit.nc

In [ ]:
#Extrapolate over land
%%bash
cd input
cdo -O fillmiss woa13_oxygen_OMIPinit.nc woa13_oxygen_OMIPinit_fillmiss.nc
cdo -O fillmiss woa13_nitrate_OMIPinit.nc woa13_nitrate_OMIPinit_fillmiss.nc
cdo -O fillmiss FEMIP_model_median_iron_2016.nc FEMIP_model_median_iron_2016_fillmiss.nc
for iv in oxygen NO3 PI_TCO2 TCO2 TAlk
do
    filename=GLODAPv2.2016b.${iv}
    cdo -O fillmiss GLODAPv2.2016b_MappedClimatologies/${filename}.nc ${filename}_fillmiss.nc
done 

In [32]:
# Apply fillmiss for `adic` and `dic` of ACCESS-OM2 OMIP2 output. 
# For adic and alk, use the 1-deg omip2 cycle 5 restart 304. 
# Because restart files have weird values over land, i cannot do `cdo fillmiss`. 
# To resolve this issue, i need to first apply the landsea mask 
# and also give appropriate grid coordinates based on an output file.
ds_in = xr.open_dataset('/g/data/ik11/outputs/access-om2/1deg_jra55_iaf_omip2_cycle5/restart304/ocean/csiro_bgc.res.nc')
ds_ou = xr.open_dataset('/g/data/ik11/outputs/access-om2/1deg_jra55_iaf_omip2_cycle5/output304/ocean/ocean_bgc_ann.nc')
ds_ou['adic'] = ds_ou['adic'].where(ds_ou['adic']!=ds_ou['adic'],ds_in['adic'].values)
ds_ou['alk'] = ds_ou['alk'].where(ds_ou['alk']!=ds_ou['alk'],ds_in['alk'].values)
ds_ou.to_netcdf('/g/data/v45/hh0162/projects/icebgc/prep_omip2/input_om2-bgc/input/1deg_jra55_iaf_omip2_cycle5_restart304_csiro_bgc.res.nc')

In [ ]:
#Extrapolate over land
%%bash
cd input
cdo -fillmiss 1deg_jra55_iaf_omip2_cycle5_restart304_csiro_bgc.res.nc 1deg_jra55_iaf_omip2_cycle5_restart304_csiro_bgc.res_fillmiss.nc


In [ ]:
#Create a reference grid file for each resolution (used as a target grid for regridding)
ncks -d time,1 -v temp /g/data/ik11/outputs/access-om2/1deg_jra55_iaf_omip2_cycle6/output365/ocean/ocean_month.nc 1deg/ocean_grid_xyz.nc
ncks -d time,1 -v temp /g/data/ik11/outputs/access-om2-025/025deg_jra55_iaf_omip2_cycle6/output365/ocean/ocean_month.nc 025deg/ocean_grid_xyz.nc
ncks -d time,1 -v temp /g/data/ik11/outputs/access-om2-01/01deg_jra55v140_iaf_cycle3/output700/ocean/ocean-3d-temp-1-daily-mean-ym_2011_01.nc 01deg/ocean_grid_xyz.nc
